In [1]:
import pandas as pd

In [2]:
import ipystata

### Import the dataset

Load all of the STATA data files (.dta)and merge on country_id (the unique contry code identifier) and year.  AFter the merge drop all observations prior to 2006

In [3]:
%%stata
use "C:\Users\klove\OneDrive\Documents\USAID\transition metrics\libdemVdem.dta", clear
sort country_id year
save "C:\Users\klove\OneDrive\Documents\USAID\transition metrics\libdemVdem.dta", replace
merge 1:1 year country_id using "C:\Users\klove\OneDrive\Documents\USAID\transition metrics\ipd for merge.dta"
rename _merge _mergeIPD
merge 1:1 year country_id using "C:\Users\klove\OneDrive\Documents\USAID\transition metrics\GDPpppfilledin.dta"
rename _merge _mergeGDP
merge 1:1 year country_id using "C:\Users\klove\OneDrive\Documents\USAID\transition metrics\biohabYale.dta"
rename _merge _mergeBioHab
merge 1:1 year country_id using "C:\Users\klove\OneDrive\Documents\USAID\transition metrics\safetysecLegatum.dta"
rename _merge _mergeSafeSecure
merge 1:1 year country_id using "C:\Users\klove\OneDrive\Documents\USAID\transition metrics\busenvLegatum.dta"
rename _merge _mergeBusEnv
merge 1:1 year country_id using "C:\Users\klove\OneDrive\Documents\USAID\transition metrics\diagaccVdem.dta"
rename _merge _mergeDiagAcc
merge 1:1 year country_id using "C:\Users\klove\OneDrive\Documents\USAID\transition metrics\socialgroupVdem.dta"
rename _merge _mergeSociGE
merge 1:1 year country_id using "C:\Users\klove\OneDrive\Documents\USAID\transition metrics\childhealthCIESIN.dta"
rename _merge _mergeChildHealth
merge 1:1 year country_id using "C:\Users\klove\OneDrive\Documents\USAID\transition metrics\exportconUNCTAD.dta"
rename _merge _mergeExpCon
merge 1:1 year country_id using "C:\Users\klove\OneDrive\Documents\USAID\transition metrics\ictWEF.dta"
rename _merge _mergeICT
merge 1:1 year country_id using "C:\Users\klove\OneDrive\Documents\USAID\transition metrics\tradefreedomHF.dta"
rename _merge _mergeTF
merge 1:1 year country_id using "C:\Users\klove\OneDrive\Documents\USAID\transition metrics\ogWJP.dta"
rename _merge _mergeOG
merge 1:1 year country_id using "C:\Users\klove\OneDrive\Documents\USAID\transition metrics\gggWEF.dta"
rename _merge _mergeGGG
merge 1:1 year country_id using "C:\Users\klove\OneDrive\Documents\USAID\transition metrics\education for merge.dta"
rename _merge _mergeEd
merge 1:1 year country_id using "C:\Users\klove\OneDrive\Documents\USAID\transition metrics\povert5dayPovCal.dta"
rename _merge _mergePoverty  
merge 1:1 year country_id using "C:\Users\klove\OneDrive\Documents\USAID\transition metrics\GovEff for merge.dta"
rename _merge _mergeGoveEff
drop income_group iso_code mrr region country_name _fillin
drop if year <2006
sort country_id year



file C:\Users\klove\OneDrive\Documents\USAID\transition metrics\libdemVdem.dt
    Result                           # of obs.
    -----------------------------------------
    not matched                         1,652
        from master                     1,512  (_merge==1)
        from using                        140  (_merge==2)

    matched                               398  (_merge==3)
    -----------------------------------------

    Result                           # of obs.
    -----------------------------------------
    not matched                           846
        from master                       278  (_merge==1)
        from using                        568  (_merge==2)

    matched                             1,772  (_merge==3)
    -----------------------------------------

    Result                           # of obs.
    -----------------------------------------
    not matched                         1,680
        from master                        25  (_merge

Make the dataset square and balanced (one observation per country for all years) then drop the fillin indicator

In [4]:
%%stata
fillin country_id year
drop _fillin

Fill in country-years without observations with previous-year country data until a new observation or until the end of the dataset.

In [5]:
%%stata
sort country_id year
by country_id: carryforward biohab, gen(biohabfwd)
by country_id: carryforward busenvlegatum , gen(busenvfwd)
by country_id: carryforward libdemvdem , gen(libdemvdemfwd)
by country_id: carryforward ipd , gen(idpfwd)
by country_id: carryforward gdpppp, gen(gdppppfwd)
by country_id: carryforward safetyseclegatum , gen( safetyseclegatumfwd )
by country_id: carryforward diagaccvdem , gen( diagaccvdemfwd )
by country_id: carryforward socialgroupvdem , gen( socialgroupvdemfwd )
by country_id: carryforward childhealthciesin , gen( childhealthciesinfwd )
by country_id: carryforward expconunctad , gen( expconunctadfwd )
by country_id: carryforward ictwef , gen( ictweffwd )
by country_id: carryforward tradefreedomhf, gen( tradefreedomhffwd )
by country_id: carryforward ogwjp, gen( ogwjpfwd )
by country_id: carryforward gggwef, gen(gggweffwd )
by country_id: carryforward education , gen(educationfwd)
by country_id: carryforward goveff , gen(govefffwd)
by country_id: carryforward headcount , gen(headcountfwd)


biohabfwd:  (180 real changes made)

busenvfwd:  (149 real changes made)

libdemvdemfwd:  (174 real changes made)

idpfwd:  (1166 real changes made)

gdppppfwd:  (427 real changes made)

safetyseclegatumfwd:  (149 real changes made)

diagaccvdemfwd:  (175 real changes made)

socialgroupvdemfwd:  (174 real changes made)

childhealthciesinfwd:  (380 real changes made)

expconunctadfwd:  (458 real changes made)

ictweffwd:  (207 real changes made)

tradefreedomhffwd:  (7 real changes made)

ogwjpfwd:  (113 real changes made)

gggweffwd:  (180 real changes made)

educationfwd:  (267 real changes made)

govefffwd:  (390 real changes made)

headcountfwd:  (969 real changes made)



Sort the combined dataset in decending order by year (i.e. -year) to facilitate filling years prior to the first observation (i..e. carrybackward)

In [6]:
%%stata
gsort country_id -year

In [7]:
%%stata
by country_id: carryforward biohabfwd , gen(biohabfill)
by country_id: carryforward educationfwd , gen(educationfill)
by country_id: carryforward headcountfwd , gen(headcountfill)
by country_id: carryforward gdppppfwd , gen(gdppppfill)
by country_id: carryforward govefffwd , gen(govefffill)
by country_id: carryforward idpfwd , gen(ipdfill)
by country_id: carryforward busenvfwd , gen(busenvfill)
by country_id: carryforward childhealthciesinfwd , gen(childhealthfill)
by country_id: carryforward diagaccvdemfwd , gen(diagaccvdemfill)
by country_id: carryforward expconunctadfwd , gen(expconunctadfill)
by country_id: carryforward gggweffwd , gen(gggweffill)
by country_id: carryforward ictweffwd , gen(ictweffill)
by country_id: carryforward libdemvdemfwd , gen(libdemvdemfill)
by country_id: carryforward ogwjpfwd , gen(opwjpfill)
by country_id: carryforward safetyseclegatumfwd , gen(safetyseclegatumfill)
by country_id: carryforward socialgroupvdemfwd , gen(socialgroupvdemfill)
by country_id: carryforward tradefreedomhffwd , gen(tradefreedomhffill)

biohabfill:  (0 real changes made)

educationfill:  (532 real changes made)

headcountfill:  (246 real changes made)

gdppppfill:  (18 real changes made)

govefffill:  (8 real changes made)

ipdfill:  (310 real changes made)

busenvfill:  (149 real changes made)

childhealthfill:  (766 real changes made)

diagaccvdemfill:  (178 real changes made)

expconunctadfill:  (231 real changes made)

gggweffill:  (250 real changes made)

ictweffill:  (637 real changes made)

libdemvdemfill:  (178 real changes made)

opwjpfill:  (832 real changes made)

safetyseclegatumfill:  (149 real changes made)

socialgroupvdemfill:  (178 real changes made)

tradefreedomhffill:  (267 real changes made)



Change North Koreas trade freedom scores from 0 to 30 to remove the outligher from the dataset and aid in standerdization

In [8]:
%%stata
replace tradefreedomhffill = 30 if country_id == 408

(13 real changes made)



Log GDP per capita (ln)

In [9]:
%%stata
gen lngdppp=ln(gdppppfill)

(585 missing values generated)



Remove observations prior to 2010 to establish 2010-most current ovservation min-maxing

In [10]:
%%stata
drop if year < 2010

(960 observations deleted)



Put all of the indicators on a "0-1" scale using $x_{rescaled}=\frac{x-x_{min}}{x_{max} -x_{min}} $

In [11]:
%%stata
norm biohabfill- lngdppp , method(mmx)

Reverse the directionality of those indicators where higher numbers are not positively correlated with capacity or commitment

In [12]:
%%stata
gen mmx_headcountdirected =1- mmx_headcountfill
gen mmx_expconddirected =1- mmx_expconunctadfill

(783 missing values generated)

(189 missing values generated)



Reorder the variables to faciliate performing batch stata comands (e.g. for $var_1-var_X$)

In [13]:
%%stata
order _mergeIPD _mergeGDP _mergeBioHab _mergeSafeSecure _mergeBusEnv _mergeDiagAcc _mergeSociGE _mergeChildHealth _mergeExpCon _mergeICT _mergeTF _mergeOG _mergeGGG _mergeEd _mergePoverty _mergeGoveEff country_id year libdemvdem ipd gdpppp biohab safetyseclegatum busenvlegatum diagaccvdem socialgroupvdem childhealthciesin expconunctad ictwef tradefreedomhf ogwjp gggwef education headcount goveff biohabfwd busenvfwd libdemvdemfwd idpfwd gdppppfwd safetyseclegatumfwd diagaccvdemfwd socialgroupvdemfwd childhealthciesinfwd expconunctadfwd ictweffwd tradefreedomhffwd ogwjpfwd gggweffwd educationfwd govefffwd headcountfwd biohabfill educationfill headcountfill gdppppfill govefffill ipdfill busenvfill childhealthfill diagaccvdemfill expconunctadfill gggweffill ictweffill libdemvdemfill opwjpfill safetyseclegatumfill socialgroupvdemfill tradefreedomhffill mmx_gdppppfill mmx_headcountfill mmx_expconunctadfill lngdppp mmx_libdemvdemfill mmx_opwjpfill mmx_socialgroupvdemfill mmx_gggweffill mmx_busenvfill mmx_tradefreedomhffill mmx_biohabfill mmx_govefffill mmx_ipdfill mmx_safetyseclegatumfill mmx_diagaccvdemfill mmx_headcountdirected mmx_educationfill mmx_childhealthfill mmx_lngdppp mmx_expconddirected mmx_ictweffill

Determine the number of metric observations available for each country year for each deminsion (capacity & commitment)

In [14]:
%%stata
egen capacitymissing=rowmiss( mmx_govefffill- mmx_ictweffill)
egen commitmissing=rowmiss( mmx_libdemvdemfill - mmx_biohabfill )

For all country years with at least 6 (out of 10) observations in the capacity demension, create a simple average from those demensions as the overall Capacity score.
For all country years with at least 5 (out of 7) observations in the commitment demension, create a simple average from those demensions as the overall Commitment score.

In [15]:
%%stata
egen capacity=rmean( mmx_govefffill- mmx_ictweffill )if capacitymissing < 5
egen commitment=rmean( mmx_libdemvdemfill - mmx_biohabfill )if commitmissing < 4

(630 missing values generated)

(630 missing values generated)



Save it an check it!!